In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer  # Explicitly enable experimental features
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import (
    StandardScaler, RobustScaler, MinMaxScaler, PowerTransformer, QuantileTransformer,
    OneHotEncoder, OrdinalEncoder, PolynomialFeatures, FunctionTransformer, KBinsDiscretizer
)
from sklearn.feature_extraction import FeatureHasher
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, SelectFromModel
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans


sns.set_theme(style="whitegrid")


In [3]:

pip install KNNImputer

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement KNNImputer (from versions: none)
ERROR: No matching distribution found for KNNImputer


In [4]:
df = pd.read_csv('Train.csv')
print(df)
df.head()

          ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0          1               D           Flight                    4   
1          2               F           Flight                    4   
2          3               A           Flight                    2   
3          4               B           Flight                    3   
4          5               C           Flight                    2   
...      ...             ...              ...                  ...   
10994  10995               A             Ship                    4   
10995  10996               B             Ship                    4   
10996  10997               C             Ship                    5   
10997  10998               F             Ship                    5   
10998  10999               D             Ship                    2   

       Customer_rating  Cost_of_the_Product  Prior_purchases  \
0                    2                  177                3   
1                    5         

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [5]:
from sklearn.impute import KNNImputer

In [6]:
if 'ID' in df.columns:
    df = df.drop('ID',axis=1)
targer_col = 'Reached.on.Time_Y.N'
x = df.drop(targer_col, axis =1)
y = df[targer_col]

print(f"Features Shape: {x.head()}")
print(f"Target Shape: {y.tail()}")

Features Shape:   Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0               D           Flight                    4                2   
1               F           Flight                    4                5   
2               A           Flight                    2                2   
3               B           Flight                    3                3   
4               C           Flight                    2                2   

   Cost_of_the_Product  Prior_purchases Product_importance Gender  \
0                  177                3                low      F   
1                  216                2                low      M   
2                  183                4                low      M   
3                  176                4             medium      M   
4                  184                3             medium      F   

   Discount_offered  Weight_in_gms  
0                44           1233  
1                59           3088  
2

In [7]:
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# --- GENERATE SYNTHETIC SHIPMENT DATA ---
# Creating a dataset that mimics the ShipmentSure problem (delays, costs, weights, etc.)
np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'Warehouse_block': np.random.choice(['A', 'B', 'C', 'D', 'F'], n),
    'Mode_of_Shipment': np.random.choice(['Flight', 'Ship', 'Road', np.nan], n, p=[0.3, 0.3, 0.3, 0.1]), # Missing values example
    'Customer_care_calls': np.random.randint(2, 7, n),
    'Customer_rating': np.random.randint(1, 6, n),
    'Cost_of_the_Product': np.random.gamma(shape=2, scale=100, size=n), # Skewed distribution
    'Prior_purchases': np.random.randint(2, 10, n),
    'Product_importance': np.random.choice(['low', 'medium', 'high', 'critical'], n, p=[0.4, 0.4, 0.15, 0.05]),
    'Gender': np.random.choice(['M', 'F'], n),
    'Discount_offered': np.random.lognormal(2, 0.5, n).astype(int), # Highly skewed, potential outliers
    'Weight_in_gms': np.random.normal(4000, 1000, n),
    'Destination_City': np.random.choice(['New York', 'London', 'Paris', 'Tokyo', 'Unknown'] + [f'City_{i}' for i in range(50)], n), # High cardinality
    'Order_Hour': np.random.randint(0, 24, n),
    'Reached_on_Time': np.random.randint(0, 2, n)
})

# Introduce Missing Values in Weight (simulating data entry errors)
df.loc[0:20, 'Weight_in_gms'] = np.nan

# Introduce Extreme Outliers in Cost (e.g. VIP shipments)
df.loc[900:910, 'Cost_of_the_Product'] = 5000 

X = df.drop('Reached_on_Time', axis=1)
y = df['Reached_on_Time']

print("✅ Synthetic Shipment Data Created.")
print(df.head(3))

✅ Synthetic Shipment Data Created.
  Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0               D              nan                    4                2   
1               F             Ship                    2                2   
2               C             Road                    6                2   

   Cost_of_the_Product  Prior_purchases Product_importance Gender  \
0           311.610821                6                low      F   
1            49.650596                3             medium      M   
2           259.168778                3                low      M   

   Discount_offered  Weight_in_gms Destination_City  Order_Hour  \
0                 3            NaN           City_3          18   
1                 7            NaN          City_29          12   
2                 7            NaN          City_46          23   

   Reached_on_Time  
0                1  
1                0  
2                0  


In [8]:
# distance = sqrt({x2-x1)2 + (y2-y1)2})
knn_imputer = KNNImputer(n_neighbors= 5)
df_knn = knn_imputer.fit_transform(df[['Weight_in_gms','Cost_of_the_Product','Discount_offered']])
print("Knn Imputation Output shape:", df_knn.shape)
type(df_knn)

Knn Imputation Output shape: (1000, 3)


numpy.ndarray

In [9]:
df_knn = pd.DataFrame(knn_imputer.fit_transform(df[['Weight_in_gms','Cost_of_the_Product','Discount_offered']]),
                      columns = ['Weight_in_gms','Cost_of_the_Product','Discount_offered'])
df_knn.describe()
df_knn

,Weight_in_gms,Cost_of_the_Product,Discount_offered
0,3473.738849,311.610821,3.0
1,3986.615693,49.650596,7.0
2,4040.379338,259.168778,7.0
3,3508.767069,244.611753,10.0
4,4204.831921,268.205824,9.0
...,...,...,...
995,574.204647,328.756075,12.0
996,2161.851553,61.328146,8.0
997,2334.039534,92.280223,4.0
998,4266.537931,193.313501,6.0


In [10]:
# Multivariant Imputation by chained Equation (MICE)

iter_imputer = IterativeImputer(max_iter=10,random_state=0)
df_mice = iter_imputer.fit_transform(df[['Weight_in_gms','Cost_of_the_Product','Discount_offered']])
print(" MICE Iputation output shape:", df_mice.shape)
df_mice

 MICE Iputation output shape: (1000, 3)


array([[4.01998672e+03, 3.11610821e+02, 3.00000000e+00],
       [4.01999232e+03, 4.96505956e+01, 7.00000000e+00],
       [4.01998784e+03, 2.59168778e+02, 7.00000000e+00],
       ...,
       [2.33403953e+03, 9.22802231e+01, 4.00000000e+00],
       [4.26653793e+03, 1.93313501e+02, 6.00000000e+00],
       [3.02866731e+03, 3.28258748e+02, 1.60000000e+01]], shape=(1000, 3))

In [11]:
# 3. Custom Transformer for Winsorization
class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower = lower_quantile
        self.upper = upper_quantile
        self.limits_ = {}

    def fit(self, X, y=None):
        for col in X.columns:
            self.limits_[col] = (X[col].quantile(self.lower), X[col].quantile(self.upper))
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in X.columns:
            low, high = self.limits_[col]
            X_copy[col] = X_copy[col].clip(lower=low, upper=high)
        return X_copy

winsorizer = Winsorizer(upper_quantile=0.95) # Cap at 95th percentile
df_capped = winsorizer.fit_transform(df[['Cost_of_the_Product']])
print(f"Max Cost Before: {df['Cost_of_the_Product'].max():.2f}, After: {df_capped['Cost_of_the_Product'].max():.2f}")

Max Cost Before: 5000.00, After: 509.34


In [12]:
def winsorize_column(series, lower=0.01, upper=0.95):
    low = series.quantile(lower)
    high = series.quantile(upper)
    return series.clip(low, high)

df['Cost_capped'] = winsorize_column(df['Cost_of_the_Product'])

print("Before max:", df['Cost_of_the_Product'].max())
print("After max:", df['Cost_capped'].max())

Before max: 5000.0
After max: 509.3400208454652


In [14]:
import numpy as np

low = df['Cost_of_the_Product'].quantile(0.01)
high = df['Cost_of_the_Product'].quantile(0.95)

df['Cost_capped'] = np.clip(df['Cost_of_the_Product'], low, high)

print(df['Cost_capped'].max())

509.3400208454652


In [15]:
print("Skewness:", df['Cost_of_the_Product'].skew())

Skewness: 8.415871414348912


In [16]:
print("Kurtosis:", df['Cost_of_the_Product'].kurt())

Kurtosis: 74.20521956410398


In [17]:
print("Count:", df['Cost_of_the_Product'].count())


Count: 1000


In [18]:
print(df.mean(numeric_only=True))

Customer_care_calls       4.047000
Customer_rating           2.950000
Cost_of_the_Product     251.914694
Prior_purchases           5.368000
Discount_offered          7.760000
Weight_in_gms          4019.987970
Order_Hour               11.790000
Reached_on_Time           0.493000
Cost_capped             199.240227
dtype: float64


In [20]:
print(df.mode().iloc[0])

Warehouse_block                 A
Mode_of_Shipment             Road
Customer_care_calls           5.0
Customer_rating               1.0
Cost_of_the_Product        5000.0
Prior_purchases               3.0
Product_importance         medium
Gender                          M
Discount_offered              5.0
Weight_in_gms          574.204647
Destination_City          City_46
Order_Hour                   18.0
Reached_on_Time               0.0
Cost_capped            509.340021
Name: 0, dtype: object


In [21]:
print(df.std(numeric_only=True))

Customer_care_calls       1.410951
Customer_rating           1.430925
Cost_of_the_Product     519.671330
Prior_purchases           2.298229
Discount_offered          4.275123
Weight_in_gms          1014.977408
Order_Hour                6.753185
Reached_on_Time           0.500201
Cost_capped             132.171028
dtype: float64


In [22]:
range_values = df.max(numeric_only=True) - df.min(numeric_only=True)
print(range_values)

Customer_care_calls       4.000000
Customer_rating           4.000000
Cost_of_the_Product    4997.766709
Prior_purchases           7.000000
Discount_offered         39.000000
Weight_in_gms          6398.870638
Order_Hour               23.000000
Reached_on_Time           1.000000
Cost_capped             493.489181
dtype: float64


In [23]:
for col in df.select_dtypes(include='number').columns:
    print(f"\nColumn: {col}")
    print("Mean:", df[col].mean())
    print("Median:", df[col].median())
    print("Std Dev:", df[col].std())


Column: Customer_care_calls
Mean: 4.047
Median: 4.0
Std Dev: 1.410950665963123

Column: Customer_rating
Mean: 2.95
Median: 3.0
Std Dev: 1.4309254164866714

Column: Cost_of_the_Product
Mean: 251.9146937103098
Median: 167.49394332840546
Std Dev: 519.6713303493506

Column: Prior_purchases
Mean: 5.368
Median: 5.0
Std Dev: 2.2982292874858814

Column: Discount_offered
Mean: 7.76
Median: 7.0
Std Dev: 4.2751230013505594

Column: Weight_in_gms
Mean: 4019.9879697713654
Median: 4047.8718866648082
Std Dev: 1014.9774077302932

Column: Order_Hour
Mean: 11.79
Median: 12.0
Std Dev: 6.753184841651051

Column: Reached_on_Time
Mean: 0.493
Median: 0.0
Std Dev: 0.5002011607355611

Column: Cost_capped
Mean: 199.24022727719537
Median: 167.49394332840546
Std Dev: 132.17102833473027
